In [1]:
import basd
import xarray as xr

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [ ]:
pr_obs_hist = xr.open_dataset('../basd/data/pr_obs-hist_fine_1979-2014.nc')
pr_sim_hist = xr.open_dataset('../basd/data/pr_sim-hist_coarse_1979-2014.nc')
pr_sim_fut = xr.open_dataset('../basd/data/pr_sim-fut_coarse_2065-2100.nc')

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [ ]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=10)
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                     'pr', params, remap_grid=True )

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [ ]:
loc = dict(lat=1, lon=1)
sim_fut_ba_loc = ba.adjust_bias_one_location(loc)

In [ ]:
pr_sim_fut_ba = ba.adjust_bias(n_jobs=-1)
#ba.save_adjustment_nc('../../../Documents/pr_sim_fut_ba.nc')

# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [ ]:
sd = basd.Downscaler(pr_obs_hist, pr_sim_fut_ba, 'pr', params)

In [ ]:
loc = dict(lat=0, lon=0)
out = sd.downscale_one_location(loc)

In [ ]:
sd.downscale()